In [1]:
import warnings

import pandas as pd

import statsmodels.api as sm
import statsmodels.formula.api as smf

import funciones_auxiliares_hito_3 as aux3
from funciones_auxiliares_hito_2 import preprocesar_y_recodificar_enunciado_uno

# warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('income-db.csv')
df_final = preprocesar_y_recodificar_enunciado_uno(df)

In [3]:
modelo = aux3.crear_modelo_logistico(df_final, 'income')
resultados = modelo.summary()
display(resultados)

Optimization terminated successfully.
         Current function value: 0.335235
         Iterations 11


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 income   No. Observations:                45222
Model:                          Logit   Df Residuals:                    45189
Method:                           MLE   Df Model:                           32
Date:                Mon, 06 Feb 2023   Pseudo R-squ.:                  0.4013
Time:                        14:02:40   Log-Likelihood:                -15160.
converged:                       True   LL-Null:                       -25322.
Covariance Type:            nonrobust   LLR p-value:                     0.000
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             -6.5398      0.452    -14.483      0.000      -7.425      -5.655
age                    0.0240      0.001     17.921      0.000       0.021       0.027
fnlwgt              7.028e-07   1.39e-07      5.049      0.000     4.3e-07    9.76e-07
gender                -0.7774      0.064    -12.161      0.000      -0.903      -0.652
capital_gain           0.0003   8.56e-06     37.240      0.000       0.000       0.000
capital_loss           0.0007   3.05e-05     21.803      0.000       0.001       0.001
hours_per_week         0.0301      0.001     22.981      0.000       0.028       0.033
federal_gov            0.4770      0.075      6.369      0.000       0.330       0.624
self_employed         -0.3148      0.042     -7.521      0.000      -0.397      -0.233
state_level_gov       -0.0243      0.045     -0.534      0.593      -0.113       0.065
unemployed            -0.9372      0.787     -1.191      0.234      -2.480       0.605
elementary_school     -1.9539      0.204     -9.583      0.000      -2.354      -1.554
high_school           -0.5908      0.037    -16.120      0.000      -0.663      -0.519
preschool             -6.5364      4.059     -1.610      0.107     -14.492       1.419
university             0.9068      0.038     23.719      0.000       0.832       0.982
divorced               0.5105      0.072      7.091      0.000       0.369       0.652
married                1.2835      0.136      9.454      0.000       1.017       1.550
separated              0.4605      0.134      3.427      0.001       0.197       0.724
widowed                0.5938      0.132      4.513      0.000       0.336       0.852
others                -0.7359      0.086     -8.510      0.000      -0.905      -0.566
white_collar           0.6698      0.035     19.075      0.000       0.601       0.739
Not_in_family         -0.9659      0.133     -7.266      0.000      -1.226      -0.705
Other_relative        -1.4649      0.186     -7.883      0.000      -1.829      -1.101
Own_child             -1.9960      0.166    -12.052      0.000      -2.321      -1.671
Unmarried             -1.2039      0.149     -8.055      0.000      -1.497      -0.911
Wife                   1.0550      0.084     12.590      0.000       0.891       1.219
Asian_Pac_Islander     0.8907      0.224      3.974      0.000       0.451       1.330
Black                  0.2909      0.190      1.531      0.126      -0.082       0.663
Other                  0.2263      0.270      0.837      0.403      -0.303       0.756
White                  0.5574      0.181      3.084      0.002       0.203       0.912
america                1.4629      0.380      3.852      0.000       0.719       2.207
asia                   1.0607      0.368      2.879      0.004       0.339       1.783
europe                 1.6626      0.393      4.228      0.000       0.892       2.433
======================================================================================
"""

In [4]:
aux3.mostrar_valores_significativos(resultados, 0.05)

Los valores que afectan significativamente (p < 0.05) a la variable dependiente son: 
|                    |       coef |   interpretacion_coef |
|:-------------------|-----------:|----------------------:|
| Intercept          | -6.5398    |            -1.63495   |
| age                |  0.024     |             0.006     |
| fnlwgt             |  7.028e-07 |             1.757e-07 |
| gender             | -0.7774    |            -0.19435   |
| capital_gain       |  0.0003    |             7.5e-05   |
| capital_loss       |  0.0007    |             0.000175  |
| hours_per_week     |  0.0301    |             0.007525  |
| federal_gov        |  0.477     |             0.11925   |
| self_employed      | -0.3148    |            -0.0787    |
| elementary_school  | -1.9539    |            -0.488475  |
| high_school        | -0.5908    |            -0.1477    |
| university         |  0.9068    |             0.2267    |
| divorced           |  0.5105    |             0.127625  |
| married     



Las variables que tienen un p-value < 0.05 y un coeficiente absoluto >= 0.4:

- gender 
- federal_gov
- divorced
- married
- separated
- widowed
- others
- white_collar
- Not_in_family
- Other_relative
- Own_child
- Unmarried
- Wife
- Asian_Pac_Islander
- White
- america
- asia
- europe

# Estrategia para el modelo predictivo:

Se realizaran 3 modelos distintos.

1. Modelo con todas las variables, independiente del p-value encontrado
2. Model con las variables con p < 0.05
3. Modelo con las variable con p < 0.05, y coeficiente > 0.4